## 1.Import packages and build a soup function

In [3]:
from bs4 import BeautifulSoup
import re
import requests
import bs4
import pandas as pd
origin = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(origin, 'lxml')

## 2.Get the table from the url

In [4]:
table = soup.find('table', class_ = "wikitable sortable")
df = pd.read_html(str(table))[0]
column_name = df.loc[[0]]
list = df.drop([0])
list.columns = column_name.iloc[0]
list.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


## 3. Ignore Borough column that is not assigned

In [5]:
# Ignore cells with a borough that is Not assigned
list1 = list[-list.Borough.isin(['Not assigned'])]
list1.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


## 4.Replace the Neighbourhood that is not assigned with Borough 

In [6]:
import numpy as np
list1['Neighbourhood'][list1['Neighbourhood'] == 'Not assigned'] = list1['Borough']
list1.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/jupyterlab/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


## 5.Duplicate process

In [7]:
group = list1.groupby(['Postcode', 'Borough'], as_index = False).agg({'Neighbourhood': ['sum']})
group.head()

,Postcode,Borough,Neighbourhood
,,,sum
0,M1B,Scarborough,RougeMalvern
1,M1C,Scarborough,Highland CreekRouge HillPort Union
2,M1E,Scarborough,GuildwoodMorningsideWest Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
print (group.columns.size)

3


In [9]:
type(group)

pandas.core.frame.DataFrame

In [10]:
group.columns = column_name.iloc[0]
print(group.head())
print(group.shape)

0 Postcode      Borough                       Neighbourhood
0      M1B  Scarborough                        RougeMalvern
1      M1C  Scarborough  Highland CreekRouge HillPort Union
2      M1E  Scarborough       GuildwoodMorningsideWest Hill
3      M1G  Scarborough                              Woburn
4      M1H  Scarborough                           Cedarbrae
(103, 3)


In [17]:
for i in group['Neighbourhood']:
    ','.join(i)

In [18]:
group.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,RougeMalvern
1,M1C,Scarborough,Highland CreekRouge HillPort Union
2,M1E,Scarborough,GuildwoodMorningsideWest Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
